In [1]:
import pennylane as qml
import numpy as np
import math
import csv

In [2]:
#random
graph = [(0,1), (0,6), (0,7), (1,2), (1,6), (2,4), (2,7), (3,7), (3,6), (4,5), (4,6), (5,1), (5,6), (5,7), (6,7)]
#ladder
#graph = [(0,1), (0,7), (1,2), (2,3), (2,7), (3,4), (3,6), (4,5), (5,6), (6,7)]
#barbell
#graph = [(0,1), (0,2), (0,3), (1,2), (1,3), (2,4), (2,3), (4,5), (4,6), (4,7), (5,6), (5,7), (6,7)]
#caveman
#graph = [(0,1), (0,2), (1,2), (1,3), (1,4), (2,3), (3,5), (4,5), (4,6), (5,6), (5,7), (6,7)]
nodes = 8


In [3]:
@qml.qnode(qml.device('default.qubit', wires=nodes, shots=1000))

def circuit(graph, nodes, inputs=None):
   for i in range(nodes):
      qml.Hadamard(wires=i)


   if inputs is not None:
          edge_1 = inputs[0]
          edge_2 = inputs[1]

   for edge in graph:
            wire1 = edge[0]
            wire2 = edge[1]
            qml.CNOT(wires = [wire1, wire2])
            qml.RZ(np.pi/2, wires=wire2 )            
            qml.CNOT(wires = [wire1, wire2])

   for i in range(nodes):
        qml.CNOT(wires=[i, (i+1)%8])

   qml.Rot(2.01, -0.18, 2.02, wires=0)
   qml.Rot(-0.43, 2.88, -1.76, wires=1)
   qml.Rot(2.39, -3.03, -2.23, wires=2)
   qml.Rot(1.86, -0.62, -0.23, wires=3)
   qml.Rot(1.03, -2.95, 0.75, wires=4)
   qml.Rot(-0.69, -2.76, 3.01, wires=5)
   qml.Rot(-1.41, -3.12, -2.37, wires=6)
   qml.Rot(1.62, 1.89, 0.77, wires=7)

   for i in range(nodes):
        qml.CNOT(wires=[i, (i+1)%8])

   qml.Rot(-1.93, 0.83, 1.78, wires=0)
   qml.Rot(-1.28, -0.35, -1.98, wires=1)
   qml.Rot(1.52, -2.71, -2.24, wires=2)
   qml.Rot(1.08, -3.04, -1.40, wires=3)
   qml.Rot(-1.26, 0.48, 1.76, wires=4)
   qml.Rot(-1.00, -1.84, 0.15, wires=5)
   qml.Rot(-0.85, 0.93, 0.79, wires=6)
   qml.Rot(-2.41, 1.55, 1.62, wires=7)

   qml.CNOT(wires=[0,1])
   qml.CNOT(wires=[3,4])
   qml.CNOT(wires=[5,6])
   qml.CNOT(wires=[7,0])

   qml.CNOT(wires=[0,1])
   qml.Rot(0.38, 1.70, 0.98, wires=2)
   qml.CNOT(wires=[3,4])
   qml.Rot(1.55, -0.97, 1.10, wires=5)
   qml.Rot(2.68, 1.71, 1.72, wires=7)

   qml.CNOT(wires=[5,6])
   qml.CNOT(wires=[7,0])

   qml.CNOT(wires=[0,1])
   qml.CNOT(wires=[2,3])
   qml.Rot(-2.50, -0.44, 2.17, wires=4)
   qml.Rot(2.10, -1.03, -1.95, wires=5)
   qml.Rot(-2.45, -0.87, -1.06, wires=6)
   qml.Rot(0.44, -0.22, -0.85, wires=7)
   qml.Rot(-1.28, 1.36, -0.06, wires=3)
   qml.CNOT(wires=[2,3])

   if inputs is None:
        return qml.sample()

   H = qml.PauliZ(edge_1) @ qml.PauliZ(edge_2)

   return qml.expval(H)   

In [4]:
def hamiltonian(config, graph):
    
    H = 0
    for i, j in graph:
        
        zi = config[i]
        zj = config[j]
        H += 0.5 * (1 - zi*zj)
    
    return H

In [5]:
def bitstring_to_int(bit_string_sample):
    # transform tensor of measurements into a list
    bit_string_sample = bit_string_sample.tolist()
    
    # join the elements of the list to make a string
    bit_string = "".join(str(bs) for bs in bit_string_sample)
    
    return int(bit_string, base=2)

In [6]:
def qnode(inputs):
    device = qml.device('default.qubit', wires=nodes, shots = 1000)
    qnode = qml.QNode(circuit, device, interface='torch')

In [7]:
def predict(inputs):
    # circuit ausführen
    return circuit(graph, nodes, inputs)

In [8]:
def run_qvc(graph):
    obj = 0
    for edge in graph:
        h_average = predict(edge)
        obj += 0.5 * (1-h_average)
    score = obj
    return score

In [9]:
def test_best_agent(graph):
    bit_strings = []
    n_samples = 100
    
    simple_measurements = []
    for i in range(0, n_samples):
        measurements = predict(None, test=True)
        simple_measurements.append(measurements)
        bit_strings.append(bitstring_to_int(measurements))
        
    samples_energies = []
    for m in simple_measurements:
        m = m.tolist()
        m = [int(x) if int(x)==1 else -1 for x in m]
        samples_energies.append(hamiltonian(m, graph))
       
    average_energy = sum(samples_energies)/len(samples_energies)
    print('average energy:', average_energy)

    counts = np.bincount(np.array(bit_strings))
    most_freq_bit_string = np.argmax(counts)
    print("Most frequently sampled bit string is: {:08b}".format(most_freq_bit_string))

In [14]:
device = qml.device('default.qubit', wires=nodes, shots = 1000)
qnode = qml.QNode(circuit, device, interface='torch')
#print(qml.draw(circuit)(graph, nodes))
